In [ ]:
import cv2
import os
import numpy as np
from skimage import color,data,restoration
from math import log10,sqrt
from scipy import signal
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse


In [ ]:
path = '/content/drive/MyDrive/Image_Processing_Plants/dataset'

folders = []
for root,dirs,files in os.walk(path):
  for dir in dirs:
    folders.append(os.path.join(root,dir))

In [ ]:
folders

['/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited',
 '/content/drive/MyDrive/Image_Processing_Plants/dataset/pests-edited',
 '/content/drive/MyDrive/Image_Processing_Plants/dataset/bacterial leaf spot-edited',
 '/content/drive/MyDrive/Image_Processing_Plants/dataset/fusarium-edited',
 '/content/drive/MyDrive/Image_Processing_Plants/dataset/healthy-edited']

In [ ]:
def PSNR(old,new):
  mse = np.mean((old-new)**2)
  if(mse == 0):
    return 100
  max_pixel = 255.0
  psnr = 20*log10(max_pixel/sqrt(mse))
  return psnr


## UnSharp Mask Filter

In [ ]:
from scipy.ndimage.filters import median_filter
def unsharp(image, sigma, strength):
  image_mf = median_filter(image, sigma)
  lap = cv2.Laplacian(image_mf,cv2.CV_64F)
  sharp = image-strength*lap
  sharp[sharp>255] = 255
  sharp[sharp<0] = 0    
  return sharp

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png')
sharp1 = np.zeros_like(image)
for i in range(3):
    sharp1[:,:,i] = unsharp(image[:,:,i], 5, 0.8)

cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/sample1.png',sharp1)
print(PSNR(image,sharp1))

42.19287075848257


# Wiener Filter

In [ ]:
def weiner_filter(image1):
  psf = np.ones((3,3),np.float32)/9
  new_image = restoration.wiener(image1,psf,1100)
  return new_image


In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (2)bg(stdres).png')
sharp1 = np.ones_like(image)
for i in range(3):
    sharp1[:,:,i] = weiner_filter(sharp1[:,:,i])

print(PSNR(image,sharp1))

34.52469183738028


# Laplacian Filter

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (2)bg(stdres).png',0)
dest = cv2.Laplacian(image, cv2.CV_16S, ksize=3)
abs_dest = cv2.convertScaleAbs(dest)
print(PSNR(image,abs_dest))
cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/laplacian.png',abs_dest)

34.85416232804122


True

# Laplacian of Gaussian Filter

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (2)bg(stdres).png',0)
gaussian_image = cv2.GaussianBlur(image,(5,5),0)
dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
abs_dest = cv2.convertScaleAbs(dest)
print(PSNR(image,abs_dest))

33.840451561021865


# Adaptive Median Filter

In [ ]:
import numpy as np
from numba import njit,prange


@njit
def padding(img,pad):
    padded_img = np.zeros((img.shape[0]+2*pad,img.shape[1]+2*pad))
    padded_img[pad:-pad,pad:-pad] = img
    return padded_img

@njit(parallel=True)
def AdaptiveMedianFilter(img,s=3,sMax=7):
    if len(img.shape) == 3:
        raise Exception ("Single channel image only")

    H,W = img.shape
    a = sMax//2
    padded_img = padding(img,a)

    f_img = np.zeros(padded_img.shape)

    for i in prange(a,H+a+1):
        for j in range(a,W+a+1):
            value = Lvl_A(padded_img,i,j,s,sMax)
            f_img[i,j] = value

    return f_img[a:-a,a:-a] 

@njit
def Lvl_A(mat,x,y,s,sMax):
    window = mat[x-(s//2):x+(s//2)+1,y-(s//2):y+(s//2)+1]
    Zmin = np.min(window)
    Zmed = np.median(window)
    Zmax = np.max(window)

    A1 = Zmed - Zmin
    A2 = Zmed - Zmax

    if A1 > 0 and A2 < 0:
        return Lvl_B(window, Zmin, Zmed, Zmax)
    else:
        s += 2 
        if s <= sMax:
            return Lvl_A(mat,x,y,s,sMax)
        else:
             return Zmed

@njit
def Lvl_B(window, Zmin, Zmed, Zmax):
    h,w = window.shape

    Zxy = window[h//2,w//2]
    B1 = Zxy - Zmin
    B2 = Zxy - Zmax

    if B1 > 0 and B2 < 0 :
        return Zxy
    else:
        return Zmed

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png',0)
new_image = AdaptiveMedianFilter(image)
print(PSNR(image,new_image))
cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/adaptive_median.png',new_image)

33.89408829412889


True

# Mid Point Filter

In [ ]:
from scipy.ndimage import maximum_filter, minimum_filter

def midpoint(img):
    maxf = maximum_filter(img, (3, 3))
    minf = minimum_filter(img, (3, 3))
    midpoint = (maxf + minf) / 2
    return midpoint

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png',cv2.IMREAD_GRAYSCALE)
new_image = midpoint(image)
print(PSNR(image,new_image))

6.765596212467145


# Contra Harmonic Mean

In [ ]:
def contraharmonic_mean(img, size, Q):
    num = np.power(img, Q + 1)
    denom = np.power(img, Q)
    kernel = np.full(size, 1.0)
    result = cv2.filter2D(num, -1, kernel) / cv2.filter2D(denom, -1, kernel)
    return result


In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png')
new_image = contraharmonic_mean(image,(3,3),0.5)
print(PSNR(image,new_image))
cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/contraharmonic_mean.png',new_image)

29.262612367501923


True

# Arithmetic Filter with Sharpening Kernel

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png')
kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])

new_image = cv2.filter2D(image, -1, kernel_sharpening)
cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/sharpKernel.png',new_image)
PSNR(image,new_image)

35.068491084056696

# Gabor Filter

In [ ]:
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
	d = K_size // 2
	gabor = np.zeros((K_size, K_size), dtype=np.float32)
	for y in range(K_size):
		for x in range(K_size):
			px = x - d
			py = y - d
			theta = angle / 180. * np.pi
			_x = np.cos(theta) * px + np.sin(theta) * py
			_y = -np.sin(theta) * px + np.cos(theta) * py
			gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)
	gabor /= np.sum(np.abs(gabor))
	return gabor



def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    H, W = gray.shape
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')
    out = np.zeros((H, W), dtype=np.float32)
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)
    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)
    return out


def Gabor_process(img):
    H, W = img.shape
    gray = img
    As = [0,30,60,90,120,150]
    out = np.zeros([H, W], dtype=np.float32)
    for i, A in enumerate(As):
        _out = Gabor_filtering(gray, K_size=11, Sigma=1.5, Gamma=1.2, Lambda=3, angle=A)
        out += _out
    out = out / out.max() * 255
    out = out.astype(np.uint8)
    return out


In [ ]:
import matplotlib.pyplot as plt
img = cv2.imread('/content/drive/MyDrive/Image_Processing_Plants/dataset/curl-edited/curl_ (1)bg(stdres).png',0)
out = Gabor_process(img)
print(PSNR(img,out))
cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/gabor.png',out)

32.01837137891787


True

# ATMED , TMED , NAFSM